### Manual Neural Networks:

In this section, we'll be creating some of the basic functions and operations of Tensorflow (Python library for Neural Networks). The purpose of this, is to get aware of the foundation of Neural Networks, and how they work. Later on, when we'll be using Tensorflow, we'll be actually understanding what's going on at the backend of those classes and function, we repeatedly use.

Following are the things we'll be coding out in Python:

#### Operation Class:

* Input nodes
* Output nodes
* Global default graph variable
* Compute method (going to be overwritten by extended class)


Alright, so without furter delay, LET'S JUMP INTO IT.

### Basic Operations:

In [39]:
class Operation():
    
    def __init__(self, input_nodes = []):
        self.input_nodes = input_nodes
        self.output_nodes = []
        
        for node in input_nodes:
            node.output_nodes.append(self)
            
        _default_graph.operations.append(self)
            
        
    def compute(self):
        pass

In [40]:
class add(Operation): # for addition of two numbers
    
    def __init__(self, x, y):
        super().__init__([x,y])
        
    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        
        return x_var + y_var

In [41]:
class multiply(Operation): # for multiplication of two numbers
    
    def __init__(self, x, y):
        super().__init__([x,y])
        
    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        
        return x_var * y_var

In [42]:
class matmul(Operation): # for matrix multiplication
    
    def __init__(self, x, y):
        super().__init__([x, y])
        
    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        
        return x_var.dot(y_var)

### Placeholders:

Placeholders are the __empty__ nodes, that need a value to be provided to compute the output. 

In [43]:
class Placeholder():
    
    def __init__(self):
        
        self.output_nodes = [] # initialize with an empty list
        _default_graph.placeholders.append(self) # append this to _default_graph object, we'll create later on!
        

### Variables:

Changeble parameters of Graph

In [44]:
class Variable():
    
    def __init__(self, initial_value = None):
        
        self.value = initial_value
        self.output_nodes = []
        
        _default_graph.variables.append(self)
        

### Graphs:

Global parameters connecting __variables__ and __placeholders__ to operations

In [45]:
class Graph(): # This is going to be connecting the 'placeholders' and 'variables' to the operations they belong to

    
    def __init__(self):
        
        self.operations = []
        self.placeholders = []
        self.variables = []
        
    def set_as_default(self):
        
        global _default_graph # going to allow us to acces this inside other classes, als keeps track of all the obove paramerters i.e operations/placeholders/variables
        _default_graph = self
        

In [46]:
# Now let's go ahead see these classes we created in action

g = Graph()

g.set_as_default()

A = Variable(10)

b = Variable(1)

x = Placeholder()

y = multiply(A,x)

z = add(y,b)

We just went ahead and initiated every single class that is supposed to perform corresponding operations But now we need to create a session class, that's going to execute this Graph class.

Now that the Graph has all the nodes, we need to execute all the operations, within a session.

For this we'll use __PostOder Tree__ traversal to make sure we execute the nodes in correct order. __Very Important!__

So, let's go ahead and create a Function for that!

In [47]:
def traverse_postorder(operation):
    """
    PostOrder Transversal of Nodes. Basically makes sure that the computations are done in correct order
    (Ax first, then Ax + b)
    """
    
    nodes_postorder = []
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
                
        nodes_postorder.append(node)
        
    recurse(operation)
    
    return nodes_postorder

In [48]:
# Session Class
import numpy as np

class Session():
    
    def run(self, operation, feed_dict = {}):
        """
        operations -> actual operations, feed_dict -> maps placeholders to conduct operations
        feed_dict{} -> later on we'll feeding data to our neural network, using this feed_dict{}
        
        """
        nodes_postorder = traverse_postorder(operation)
        
        for node in nodes_postorder:
            
            if type(node) == Placeholder: # if node type is Placeholder, pass in as a key for feed_dict{}
                
                node.output = feed_dict[node]
                
            elif type(node) == Variable:
                
                node.output = node.value
                
            else:
                # OPERATION
                node.inputs = [input_node.output for input_node in node.input_nodes]
                
                node.output = node.compute(*node.inputs) # since we don't know how many arguments we're gonna
                # pass in, so we're using *(args) here. because the size of the list could be unknown!

            if type(node.output) == list:
                node.output = np.array(node.output)
                
        return operation.output

In [49]:
sess = Session()

In [50]:
result = sess.run(operation = z, feed_dict = {x: 10})

In [52]:
result # reports back the reqeusted operation!

101

In [56]:
# Now let's show some matrix multiplications!

g = Graph()

g.set_as_default()

A = Variable([[10,20],[30,40]])

b = Variable([1,1])

x = Placeholder()

y = matmul(A,b)

z = add(y,b)

In [58]:
sess = Session()

In [59]:
result = sess.run(operation = z, feed_dict = {x: 10})

In [61]:
result 

array([ 51, 112])